# Estructuras básicas de datos

En este capítulo veremos cómo implementar en Swift algunas estructuras de datos fundamentales.

Las **estructuras de datos** son abstracciones que nos permiten almacenar datos siguiendo unos parámetros conocidos en cuanto al uso de las colecciones y el consumo de memoria asociado a las mismas. Una vez conocemos cómo funciona una lista enlazada, una pila o una cola, podemos utilizarlas para diferentes propósitos.

Las estructuras de datos son conceptos generales que pueden utilizarse en cualquier lenguaje de programación, de modo que el conocimiento de su funcionamiento es esencial independientemente del lenguaje que estemos utilizando.

1. [Listas enlazadas](#Listas-enlazadas)
2. [Pilas (_Stacks_)](#Pilas-(Stacks))
3. [Genéricos](#Gen%C3%A9ricos)
4. [Colas (_Queues_)](#Colas-(Queues))

## Listas enlazadas

Una lista enlazada (_linked list_) está formada por un conjunto de nodos. Cada nodo contiene dos elementos:
* Un **valor**. Es el dato que queremos almacenar.
* Una referencia al **siguiente nodo**.

![Lista enlazada](img/linked-list.png "Ejemplo de lista enlazada de enteros")

En Swift podemos modelar un **nodo** como una clase con esos dos campos. La siguiente celda define un nodo para almacenar valores de tipo `String`.

In [ ]:
class Node {
    var value: String
    var next: Node?
    var previous: Node?
    
    init(value: String, next: Node? = nil, previous: Node? = nil) {
        self.value = value
        self.next = next
        self.previous = previous
    }
}

Utilizando el concepto de nodo podemos construir una lista:

In [ ]:
let n1 = Node(value: "urjc")
let n2 = Node(value: "stanford")
let n3 = Node(value: "mit")

In [ ]:
n1.next = n2
n2.next = n3

La celda anterior establece que el _siguiente_ a `n1` es `n2`. A su vez, el nodo que sigue a `n2` as `n3`.

Vamos a implementar `description` para verlo más claro. Cuando hacemos `print` de un nodo mostramos su valor y, si tiene `next`, mostramos el siguiente nodo:

In [ ]:
extension Node : CustomStringConvertible {
    public var description: String {
        guard let next = next else { return "[\(value)]" }
        return "[\(value)] -> \(next)"
    }
}

In [ ]:
print(n1)

La manipulación manual de nodos es prolija y propensa a error. El paso natural es **encapsular** algunas operaciones básicas dentro de un tipo especial que llamaremos `LinkedList`. Una lista enlazada viene definida fundamentalmente por la _cabeza_ de la lista. Si tenemos una referencia al primer nodo, podemos acceder a toda la lista.

In [ ]:
struct LinkedList {
    var head: Node?
}

In [ ]:
extension LinkedList : CustomStringConvertible {
    public var description: String {
        guard let head = head else { return "Empty list." }
        return "\(head)"
    }
}

In [ ]:
var lista = LinkedList(head: n1)

In [ ]:
print(lista)

Con esto ganamos poco, seguimos teniendo que manipular los nodos manualmente. Vamos a incorporar una función para añadir nodos **al final de la lista**. Además, en lugar de construir los nodos podemos dar simplemente los valores y que se encargue de construirlos la propia lista.

Recordemos que nuestros nodos están diseñados por el momento para almacenar `String`s.

### Añadiendo nodos al final de la lista

In [ ]:
extension LinkedList {
    func findLast() -> Node? {
        guard let head = head else { return nil }
        var node = head
        while node.next != nil {
            node = node.next!
        }
        return node
    }
    
    mutating func append(_ value: String) {
        let newNode = Node(value: value)
        if let lastNode = findLast() {
            lastNode.next = newNode
        } else {
            head = newNode
        }
    }
}

In [ ]:
var lista = LinkedList()

In [ ]:
print(lista)

In [ ]:
lista.append("a")
lista.append("b")
lista.append("c")
print(lista)

Funciona, pero hay un problema de eficiencia: **estamos recorriendo la lista entera cada vez que queremos añadir un elemento**.

¿Cómo podemos solucionarlo?

Una forma es guardar en una variable el último elemento de la lista. Igual que tenemos una variable `head` que apunta al primero, utilizaremos `tail` para apuntar al último.

En las extensiones no podemos añadir variables, así que tenemos que repetir la definición de `LinkedList`. La siguiente definición sustituye a la dada anteriormente. Vamos a aprovechar para que tanto `head` como `tail` sean privadas, de modo que no puedan ser manipuladas accidentalmente por nadie, excepto la propia lista. Utilizaremos un mecanismo especial, **`private(set)`**, que significa que la variable correspondiente no se puede modificar, pero **sí se puede consultar**. Esto nos permitirá comprobar que estamos construyendo la lista correctamente.

Por último, crearemos un inicializador "vacío" para que todos los elemenos se vayan añadiendo utilizando la función prevista para ello.

In [ ]:
struct LinkedList {
    private(set) var head: Node?
    private(set) var tail: Node?
    
    init() {}
}

Ahora es nuestra responsabilidad que `tail` apunte en todo momento al último nodo de la cadena.

In [ ]:
extension LinkedList {    
    mutating func append(_ value: String) {
        // Create node
        let newNode = Node(value: value)
        
        // The tail, if it exists, must point to the new node
        if let tail = tail {
            tail.next = newNode
        }
        
        // Update tail and head
        tail = newNode
        if head == nil { head = newNode }
    }
}

In [ ]:
// Mismo que antes. Tenemos que repetirlo porque hemos reemplazado la definición de LinkedList
extension LinkedList : CustomStringConvertible {
    public var description: String {
        guard let head = head else { return "Empty list." }
        return "\(head)"
    }
}

In [ ]:
var lista = LinkedList()
lista.append("a")
lista.append("b")
lista.append("c")
print(lista)

In [ ]:
print(lista.tail)

### Insertando nodos al principio de la lista

Por similitud con otros tipos de datos, como las pilas, esta operación suele llamarse **`push`**.

Su implementación es muy sencilla: el nuevo nodo se convierte en la _cabeza_ de la lista.

In [ ]:
extension LinkedList {
    mutating func push(_ value: String) {
        let newNode = Node(value: value, next: head)
        head = newNode
        if tail == nil { tail = newNode }
    }
}

In [ ]:
var lista = LinkedList()

In [ ]:
lista.push("julia")
print(lista.tail)

In [ ]:
lista.push("jorge")
print(lista.tail)

In [ ]:
print(lista)

Refactorizamos para hacer más sencillo:

In [ ]:
extension LinkedList {
    mutating func push(_ value: String) {
        head = Node(value: value, next: head)
        if tail == nil { tail = head }
    }
}

In [ ]:
var lista = LinkedList()
lista.append("c")
lista.push("b")
lista.push("a")
lista.append("z")
print(lista)

In [ ]:
print(lista.tail)

### Iteración

Vamos a introducir una función para obtener el valor del nodo en una posición determinada, y así poder utilizar la lista en bucles. Llamaremos a esta función `value(at:)`:

In [ ]:
extension LinkedList {
    func value(at index: Int) -> String? {
        var i = 0
        var node = head
        while node != nil && i < index {
            node = node!.next
            i = i+1
        }
        return node?.value
    }
}

In [ ]:
print(lista.value(at: 0))
print(lista.value(at: 1))

In [ ]:
print(lista.value(at: -8))

-----
**Ejercicio 1**: Arregla el ejemplo anterior.

In [ ]:
extension LinkedList{
    func value(at index: Int) -> String? {
        var i = 0
        var node = head
        while node != nil && i < abs(index) {
            node = node!.next
            i = i+1
        }
        return node?.value
    }
}

**Ejercicio 2**: Introduce una propiedad `count` que indique cuántos elementos están almacenados en la lista. Puedes hacerlo de dos maneras:
* Contándolos cada vez (fácil, pero poco eficiente).
* Manteniendo una variable con la suma total. Debes actualizar para ello el código de todas las funciones que insertan nodos.

In [ ]:
extension LinkedList{
   var count: Int {
        guard var node = head else {
            return 0
        }
        //Si pasa el guard ya hay 1 nodo
        var count = 1
       // Suma
        while let next = node.next {
            node = next
            count += 1
        }
        return count
    }
}   

In [ ]:
var totalNodes = lista.count
print(totalNodes)

Si tu implementación es correcta, el siguiente bucle debe funcionar y mostrar todos los elementos:

In [ ]:
for i in 0..<lista.count {
    print(lista.value(at: i))
}

----

Vamos a refactorizar de nuevo. Para obtener el valor en una posición vamos a utilizar otra función que nos de **el nodo** en esa posición. Esto lo hacemos porque luego querremos insertar nodos en posiciones arbitrarias, y la nueva función nos será de utilidad.

In [ ]:
extension LinkedList {
    func node(at index: Int) -> Node? {
        var i = 0
        var node = head
        while node != nil && i < index {
            node = node!.next
            i = i+1
        }
        return node
    }
    
    func value(at index: Int) -> String? {
        return node(at: index)?.value
    }
}

In [ ]:
print(lista.value(at: 0))
print(lista.value(at: 1))

### Insertando un nodo en una posición arbitraria de la lista

Para realizar esta operación tenemos que tener cuidado de manipular las referencias a los nodos correctamente.

![Antes de la inserción](img/linked-list-insert-before.png "Inserción: antes")
![Después de la inserción](img/linked-list-insert-after.png "Inserción: después")

In [ ]:
extension LinkedList {
    mutating func insert(_ value: String, at index: Int) {
        guard index > 0 else {
            push(value)
            return
        }
        var prev = node(at: index-1)
        let node = Node(value: value, next: prev?.next)
        prev?.next = node
        if node.next == nil { tail = node }
    }
}

In [ ]:
print(lista)

In [ ]:
lista.insert("d", at: 3)
print(lista)

In [ ]:
lista.insert("0", at: 0)
print(lista)

In [ ]:
lista.insert("ZZ", at: 6)
print(lista)

In [ ]:
print(lista.tail)

### Borrar nodos

Como en el caso de la inserción, podemos hacer varias funciones con el mismo propósito:
* `removeLast` para eliminar el último. Es la contrapartida de `append`.
* `pop`, para eliminar el primero. Es la contrapartida de `push`.
* `remove(at:)` para eliminar un nodo en una posición concreta. Es la contrapartida de `insert(at:)`.

**En todos los casos debemos tener cuidado de manipular los punteros correctamente.**

Adicionalmente, vamos a hacer que las funciones de borrado devuelvan el elemento que se acaba de eliminar. Esto es útil en muchos casos, en los que queremos hacer algo con un elemento de la lista y después quitarlo.

In [ ]:
extension LinkedList {
    mutating func pop() -> String? {
        let node = head
        head = head?.next
        return node?.value
    }
}

In [ ]:
print(lista)

In [ ]:
print(lista.pop())

In [ ]:
print(lista)

**Ejercicio 3**: Implementa `removeLast` y `remove(at:)`.

In [ ]:
extension LinkedList {
    mutating func remove(node: Node) -> String? {
        let prev = node.previous
        let next = node.next
        
        if let prev = prev {
             prev.next = next
        }else{
            //Si intentamos eliminar head
             head = next
        }
        next?.previous = prev
        
        if next == nil {
            tail = prev
        }
        
        //Porque siguen en memoria y borramos el nodo
        node.previous = nil
        node.next = nil
        
        
        return node.value
    }
    
       
    
    mutating func removeLast() -> String? {
        
        return remove(node: tail!)
    }
    
    mutating func remove(at index: Int) -> String? {
        let node = self.node(at: index)!
        return remove(node: node)
    }    
   
}

----

## Pilas (_Stacks_)

La pila (_stack_) es otra estructura de datos que se utiliza muchísimo. Sigue el convenio de acceso denominado **LIFO** (_Last-In First-Out_): el último elemento que entra en la lista es el primero que se procesa. Esto sirve para modelar problemas donde tenemos que procesar los elementos en orden inverso a su llegada. Por ejemplo, un navegador web podría ir guardando las páginas visitadas en una pila, y el botón de "Atrás" del navegador iría a la página visitada inmediatamente antes (que fue la última que entró en la pila).

Las pilas sólo tienen dos funciones básicas: `push()`, para introducir un elemento, y `pop()` para extraerlo.

Vamos a implementar una pila apoyándonos para ello en un _array_.

In [ ]:
struct StringStack {
    private var storage = [String]()
    
    mutating func push(_ value: String) {
        storage.insert(value, at: 0)
    }
    
    mutating func pop() -> String? {
        guard storage.count > 0 else { return nil }
        return storage.remove(at: 0)
    }
}

In [ ]:
var webStack = StringStack()
webStack.push("urjc.es")
webStack.push("stanford.edu")
webStack.push("mit.edu")

In [ ]:
print(webStack.pop())

**Ejercicio 4**: Implementa `StringStack` utilizando una lista enlazada como almacenamiento.

In [ ]:
struct StringStack {
    private var storage = LinkedList()
    
    mutating func push(_ value: String) {
        storage.push(value)
    }
    
    mutating func pop() -> String? {
        return storage.pop()
    }
}

## Genéricos

Las estructuras que hemos estado viendo hasta ahora sólo nos permiten almacenar valores de tipo `String`. ¿Qué ocurre si quisiéramos hacer una lista de `Int`s? ¿O de cualquier otro tipo arbitrario?

* Los **algoritmos** de inserción, iteración y borrado son **exactamente iguales**.
* Sin embargo, tendríamos que repetir el código para indicar el tipo que estamos almacenando. Cambiaríamos todos los `String` por `Int`.

**Ejemplo** Comparación de una pila de `String` y una pila de `Int`:

![Dos pilas](img/two-stacks.png "Pila de String vs Int")

Repetir esencialmente **el mismo código** todo el rato es una mala idea.

Para resolverlo, usamos el mecanismo de programación _genérica_, en el que **parametrizamos el tipo que se almacena en la estructura de datos**.

In [ ]:
struct Stack<Element> {
    private var storage = [Element]()
    
    mutating func push(_ value: Element) {
        storage.insert(value, at: 0)
    }
    
    mutating func pop() -> Element? {
        guard storage.count > 0 else { return nil }
        return storage.remove(at: 0)
    }
}

En lugar de **especificar el tipo**, utilizamos un tipo simbólico entre ángulos. En este ejemplo, lo hemos llamado **`<Element>`**, pero podríamos haber utilizado otro nombre entre ángulos.

No podemos utilizar el `Stack` a menos que **lo particularicemos** para un tipo concreto. Por ejemplo, la siguiente celda crea un `Stack` particularizado para el caso de `String`s:

In [ ]:
var webStack = Stack<String>()
webStack.push("urjc.es")
webStack.push("stanford.edu")
webStack.push("mit.edu")

Del mismo modo, la siguiente celda crea un `Stack` particularizado para almacenar `Int`s:

In [ ]:
var intStack = Stack<Int>()
intStack.push(7)
intStack.push(13)
intStack.push(42)

In [ ]:
print(intStack.pop())

**Ejercicio 5** Crea una lista enlazada genérica utilizando el código que hemos visto para `LinkedList`.

In [ ]:
class Node<Element> {
    var value: Element
    var next: Node?
    var previous: Node?
    
    init(value: Element, next: Node? = nil, previous: Node? = nil) {
        self.value = value
        self.next = next
        self.previous = previous
    }
}

extension Node : CustomStringConvertible {
    public var description: String {
        guard let next = next else { return "[\(value)]" }
        return "[\(value)] -> \(next)"
    }
}

struct LinkedList<Element> {
    private(set) var head: Node<Element>?
    private(set) var tail: Node<Element>?
    
    init() {}
}

extension LinkedList : CustomStringConvertible {
    public var description: String {
        guard let head = head else { return "Empty list." }
        return "\(head)"
    }
}

extension LinkedList {    
    mutating func append(_ value: Element) {
       
        let newNode = Node(value: value)
        
        
        if let tail = tail {
            tail.next = newNode
        }
        
        
        tail = newNode
        if head == nil { 
            head = newNode 
        }
    }

    mutating func push(_ value: Element) {
        head = Node(value: value, next: head)
        if tail == nil { tail = head }
    }

    func node(at index: Int) -> Node<Element>? {
        var i = 0
        var node = head
        while node != nil && i < index {
            node = node!.next
            i = i+1
        }
        return node
    }
    
    func value(at index: Int) -> Element? {
        return node(at: index)?.value
    }
    
    var count: Int {
        guard var node = head else {
            return 0
        }
        //Si pasa el guard ya hay 1 nodo
        var count = 1
       // Suma
        while let next = node.next {
            node = next
            count += 1
        }
        return count
    }

    mutating func insert(_ value: Element, at index: Int) {
        guard index > 0 else {
            push(value)
            return
        }
        var prev = node(at: index-1)
        let node = Node(value: value, next: prev?.next)
        prev?.next = node
        if node.next == nil { tail = node }
    }

    mutating func pop() -> Element? {
        let node = head
        head = head?.next
        return node?.value
    }
    
         
    
    
}

-----

Esta es mi implementación (incompleta, faltan algunas funciones). **Si no has hecho el ejercicio intenta terminarlo antes de mirarla, o no te servirá para aprender**. 

-----

In [ ]:
class Node<T> {
    var value: T
    var next: Node?
    
    init(value: T, next: Node? = nil) {
        self.value = value
        self.next = next
    }
}

extension Node : CustomStringConvertible {
    public var description: String {
        guard let next = next else { return "[\(value)]" }
        return "[\(value)] -> \(next)"
    }
}

In [ ]:
struct LinkedList<T> {
    private(set) var head: Node<T>?
    private(set) var tail: Node<T>?
    
    init() {}
}

extension LinkedList : CustomStringConvertible {
    public var description: String {
        guard let head = head else { return "Empty list." }
        return "\(head)"
    }
}

extension LinkedList {    
    mutating func append(_ value: T) {
        // Create node
        let newNode = Node(value: value)
        
        // The tail, if it exists, must point to the new node
        if let tail = tail {
            tail.next = newNode
        }
        
        // Update tail and head
        tail = newNode
        if head == nil { head = newNode }
    }

    mutating func push(_ value: T) {
        head = Node(value: value, next: head)
        if tail == nil { tail = head }
    }

    func node(at index: Int) -> Node<T>? {
        var i = 0
        var node = head
        while node != nil && i < index {
            node = node!.next
            i = i+1
        }
        return node
    }
    
    func value(at index: Int) -> T? {
        return node(at: index)?.value
    }

    mutating func insert(_ value: T, at index: Int) {
        guard index > 0 else {
            push(value)
            return
        }
        var prev = node(at: index-1)
        let node = Node(value: value, next: prev?.next)
        prev?.next = node
        if node.next == nil { tail = node }
    }

    mutating func pop() -> T? {
        let node = head
        head = head?.next
        return node?.value
    }
}


   
       
  
   


El mecanismo de genéricos es ampliamente utilizado en los tipos estándar de Swift, aunque a veces no lo veamos. Recuerda que, [cuando vimos los arrays](00_swift_overview_class.ipynb#Arrays), comentamos que la forma canónica de expresar su tipo era `Array<TipoElemento>`.

### Genéricos y Protocolos

En el ejemplo anterior hemos visto cómo hacer una pila genérica, lo que nos permite utilizarla para almacenar cualquier tipo que necesitemos.

También hemos visto que podemos escribir diferentes implementaciones de una misma estructura según los requisitos de memoria y velocidad que necesitemos. Por ejemplo, podemos hacer una pila con un array, o con una lista enlazada. A la hora de _utilizar_ la pila, nos debería resultar indiferente _cómo_ está implementada.

Para poder facilitar esta separación entre el _qué_ debe hacer un objeto y el _cómo_ lo hace es para lo que se usan los **protocolos**. En esta sección veremos cómo hacer protocolos que, además, son **genéricos**.

Tal y como hemos definido las pilas, son colecciones que permiten dos operaciones básicas: `push` y `pop`. Podemos definir un **protocolo** que especifica estas dos operaciones. Cualquier pila, la hagamos como la hagamos, debe cumplir ese protocolo:

In [ ]:
protocol Stack {
    associatedtype Element
    
    mutating func push(_ value: Element)
    mutating func pop() -> Element?
}

En el caso de los protocolos, la forma de hacerlos genéricos es utilizando **`associatedtype`**, con el mismo propósito con el que empleamos `<Element>` en el caso de estructuras y clases. Estamos diciendo que el tipo `Element`, asociado al protocolo, es una "plantilla" para el tipo real que luego utilizaremos.

Cualquier tipo que verifique las condiciones del protocolo puede usarse como un `Stack`. Cuando escribimos una implementación que cumple el protocolo, el tipo que utilizamos en esa implementación es el que se *asocia* al `associatedtype` del protocolo.

La siguiente implementación es un stack de `Int`. Swift se da cuenta de que lo que en el protocolo llamábamos **`Element`**, ahora se particulariza para **`Int`**:

In [ ]:
struct IntStack : Stack {
    private var storage = [Int]()
    
    mutating func push(_ value: Int) {
        storage.insert(value, at: 0)
    }
    
    mutating func pop() -> Int? {
        guard storage.count > 0 else { return nil }
        return storage.remove(at: 0)
    }
}

El tipo genérico del protocolo se ha asociado a `Int`, para el caso de `IntStack`.
* **`Stack.Element`** se asocia a **`Int`**.

Pero además, el tipo al que asociamos el protocolo puede, a su vez, ser genérico. A continuación repetimos nuestra implementación genérica de `Stack`, que simplemente hemos renombrado para evitar confusiones:

In [ ]:
struct ArrayStack<Element> : Stack {
    private var storage = [Element]()
    
    mutating func push(_ value: Element) {
        storage.insert(value, at: 0)
    }
    
    mutating func pop() -> Element? {
        guard storage.count > 0 else { return nil }
        return storage.remove(at: 0)
    }
}

En este caso, **el tipo asociado al protocolo es el tipo genérico `<Element>`**. Este stack sigue siendo genérico, tenemos que particularizarlo cuando lo vayamos a usar, como vimos anteriormente.

El hecho de haber utilizado `Element` es casual. NO tienen por qué tener el mismo nombre. Por ejemplo, podríamos haber escrito el `struct` así:

In [ ]:
struct ArrayStack<T> : Stack {
    private var storage = [T]()
    
    mutating func push(_ value: T) {
        storage.insert(value, at: 0)
    }
    
    mutating func pop() -> T? {
        guard storage.count > 0 else { return nil }
        return storage.remove(at: 0)
    }
}

* Protocolo: `associatedtype Element`.
* `struct ArrayStack`: tipo genérico `<T>`.
* `ArrayStack` cumple el protocolo `Stack`
* **`Stack.Element`** se asocia a **`<T>`**, que es el tipo de `ArrayStack`.

Si tenemos una función que procesa todos los elementos de una pila, esa función **no necesita saber** cómo está implementada la pila, ni cuáles son los tipos que almacena. Pero como el protocolo `Stack` es _genérico_, nuestra función también debe serlo: depende del tipo que se esté considerando en cada momento.

Por ejemplo:

In [ ]:
func processAll<S: Stack>(stack: inout S) {
    var item: S.Element?
    repeat {
        item = stack.pop()
        if item != nil {
            print(item!)
        }
    } while item != nil
}

* Esta función es capaz de imprimir los elementos de cualquier pila, independientemente de cómo esté implementada y qué tipo de datos almacene.
* La función es genérica, pues el tipo de la pila no lo conocemos hasta que no la usemos.
* Podemos referirnos a ese tipo desconocido como **`S.Element`**, siendo **`S`**  un nombre simbólico que damos al tipo concreto que recibiremos.

In [ ]:
var words = ArrayStack<String>()  // Creamos pila de Strings

In [ ]:
words.push("the")
words.push("rain")
words.push("in")
words.push("Spain")

In [ ]:
processAll(stack: &words)

Cuando llamamos a `processAll` con una pila en concreto, ya no tenemos que poner ángulos ni nada. Lo que sucede es:
* **`S`** se refiere a **`ArrayStack<String>`**.
* **`S.Element`** se asocia a **`String`**.

In [ ]:
print(words.pop())

Del mismo modo que hemos escrito una pila genérica que utiliza un array como almacenamiento, podemos escribir una versión, también genérica, que utiliza una lista enlazada para guardar los datos. La siguiente versión utiliza una implementación (incompleta) de lista enlazada genérica:

In [ ]:
struct ListStack<T> : Stack {
    private var storage = LinkedList<T>()
    
    mutating func push(_ value: T) {
        storage.push(value)
    }
    
    mutating func pop() -> T? {
        return storage.pop()
    }
}

In [ ]:
var numbers = ListStack<Int>()

In [ ]:
for n in 1..<5 { numbers.push(n) }

In [ ]:
processAll(stack: &numbers)

En este caso:
* **`S`** se refiere a **`ListStack<Int>`**.
* **`S.Element`** se asocia a **`Int`**.

## Colas (_Queues_)

Las colas son otro ejemplo de estructura de datos ampliamente utilizada. En la vida diaria estamos familiarizados con las colas: para comprar en la pescadería, para entrar en un concierto, para entrar por carretera en Madrid.

A diferencia de las pilas, las colas siguen un orden de acceso llamado **FIFO** (_First-In First-Out_), es decir: el primero que llega es el primero que sale.

Se usan colas para procesar elementos en el mismo orden que se fueron almacenando. Por ejemplo: mensajes que llegan de la red. Los trabajos asíncronos que estamos utilizando en la práctica se implementan también con colas de ejecución.

Las colas tienen dos funciones básicas:
* `enqueue`, para introducir un nuevo elemento. Este nuevo elemento se incorpora "al final" de la cola.
* `dequeue`, para sacar un elemento. El elemento que sale es el más antiguo de los que entraron; es decir, está "el primero" de la fila.

Como ya sabemos un montón de genéricos y protocolos, vamos a definir estas funciones básicas en un protocolo al que llamaremos `Queue`. Además de las funciones básicas, vamos a modelar dos operaciones de consulta adicionales:
* `isEmpty`: nos indica si hay algún elemento en la cola.
* `peek`: nos permite consultar cuál es el siguiente elemento a procesar, sin extraerlo.

In [ ]:
protocol Queue {
    associatedtype Element
    
    mutating func enqueue(_ value: Element)
    mutating func dequeue() -> Element?
    
    var isEmpty: Bool { get }
    func peek() -> Element?
}

Observa cómo se especifican las variables o propiedades en los protocolos. En este caso, se ha declarado `isEmpty` como una propiedad **de sólo lectura** (sólo tiene `get`).

**Ejercicio 6**: Implementa un `Queue` genérico utilizando un array como almacenamiento. Se proporcionan algunas celdas que te permitirán verificar si el funcionamiento básico es correcto, pero debes probarlo exhaustivamente.

In [ ]:
struct ArrayQueue<T> : Queue {
    private var storage = [T]()
    
    mutating func enqueue(_ value: T) {
        // Your code here
         storage.append(value)
    }
    
    mutating func dequeue() -> T? {
        // Your code here
        if storage.isEmpty {
            return nil
            
        }else{
            
            let temp = storage.first
            storage.remove(at: 0)
            return temp
        }
    }
    
    var isEmpty: Bool {
        // Your code here
        return storage.isEmpty
        
    }
    
    func peek() -> T? {
        // Your code here
        //Devuelve el primer elemento en la cola
        // sin borrarlo
        return storage.first?.value
        
    }
}

La dos celdas siguientes no deberían mostrar ningún mensaje si tu implementación es correcta (esto no quiere decir que no exista algún otro error que no se esté probando).

In [ ]:
func testQueue() {
    var q = ArrayQueue<String>()
    if !q.isEmpty { print("isEmpty should be true when the queue is empty") }
    if q.dequeue() != nil { print("dequeue should return nil on an empty queue") }
    if q.peek() != nil { print("peek should return nil on an empty queue") }
    
    ["the", "rain", "in", "Spain"].forEach { q.enqueue($0) }
    
    var v = q.peek()
    if v != "the" { print("peek error, should return 'the' but your implementation returned \(v)") }
    v = q.peek()
    if v != "the" { print("peek error, should return 'the' but your implementation returned \(v)") }

    v = q.dequeue()
    if v != "the" { print("dequeue error, should return 'the' but your implementation returned \(v)") }

    q.dequeue()  // rain
    q.dequeue()  // in
    
    v = q.peek()  // Spain
    if v != "Spain" { print("peek error, should return 'Spain' but your implementation returned \(v)") }

    v = q.dequeue()
    if v != "Spain" { print("dequeue error, should return 'Spain' but your implementation returned \(v)") }

    if q.dequeue() != nil { print("dequeue error, should return nil") }
    if q.peek() != nil { print("peek error, should return nil") }
    if !q.isEmpty { print("isEmpty should be true when the queue is empty") }

    q.enqueue("test")
    if q.isEmpty { print("queue should not be empty") }
}

In [ ]:
testQueue()

**Ejercicio 7**: Implementa un `Queue` genérico utilizando una lista enlazada como almacenamiento. Pruébalo de forma similar a como hemos hecho en el caso anterior.

In [ ]:
struct ListQueue<T> : Queue {
    private var storage = LinkedList<T>()
    
    mutating func enqueue(_ value: T) {
        // Your code here
         storage.append(T)
    }
    
    mutating func dequeue() -> T? {
        // Your code here
         if storage.isEmpty {
            return nil
            
        }else{
            
            let temp = storage.first
            storage.remove(at: 0)
            return temp
        }

        return storage.value
    }
    
    var isEmpty: Bool {
        // Your code here
        return storage.isEmpty
    }
    
    func peek() -> T? {
        // Your code here
        return storage.first?.value
    }
}

In [ ]:
func testListQueue() {
    var q = ListQueue<String>()
    if !q.isEmpty { print("isEmpty should be true when the queue is empty") }
    if q.dequeue() != nil { print("dequeue should return nil on an empty queue") }
    if q.peek() != nil { print("peek should return nil on an empty queue") }
    
    ["the", "rain", "in", "Spain"].forEach { q.enqueue($0) }
    
    var v = q.peek()
    if v != "the" { print("peek error, should return 'the' but your implementation returned \(v)") }
    v = q.peek()
    if v != "the" { print("peek error, should return 'the' but your implementation returned \(v)") }

    v = q.dequeue()
    if v != "the" { print("dequeue error, should return 'the' but your implementation returned \(v)") }

    q.dequeue()  // rain
    q.dequeue()  // in
    
    v = q.peek()  // Spain
    if v != "Spain" { print("peek error, should return 'Spain' but your implementation returned \(v)") }

    v = q.dequeue()
    if v != "Spain" { print("dequeue error, should return 'Spain' but your implementation returned \(v)") }

    if q.dequeue() != nil { print("dequeue error, should return nil") }
    if q.peek() != nil { print("peek error, should return nil") }
    if !q.isEmpty { print("isEmpty should be true when the queue is empty") }

    q.enqueue("test")
    if q.isEmpty { print("queue should not be empty") }
}

In [ ]:
testListQueue()